In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
!pip install kaggle

!kaggle datasets download -d sshikamaru/udacity-self-driving-car-dataset


In [ ]:
dataset_path = '/Users/Pauline/autonomous-vehicle-classifier/udacity-self-driving-car-dataset/data/export'
annotatinos_path = '/Users/Pauline/autonomous-vehicle-classifier/udacity-self-driving-car-dataset/data/export/_annotations.csv'

In [ ]:
annotations_path = os.path.join(dataset_path, '_annotations.csv')
annotations = pd.read_csv(annotations_path)

annotations.head()


In [ ]:
group_annotations = annotations.groupby('filename')
image_file = ''